In [5]:

import numpy as np
import tensorflow as tf
import midi_statistics
import utils
import os
from gensim.models import Word2Vec


In [6]:

syll_model_path = './enc_models/syllEncoding_20190419.bin'
word_model_path = './enc_models/wordLevelEncoder_20190419.bin'


In [7]:

syllModel = Word2Vec.load(syll_model_path)
wordModel = Word2Vec.load(word_model_path)

'''
lyrics = [['Must','Must'],['have','have'],['been','been'],['love','love'],
          ['but','but'],['its','its'],['o','over'],['ver','over'],['now','now'],['lay','lay'],['a','a'],
          ['whis','whisper'],['per','whisper'],['on','on'],['my','my'],['pil','pillow'],['low','pillow']]
lyrics = [['E','Everywhere'],['very','Everywhere'],['where','Everywhere'],['I','I'],['look','look'],
         ['I','I'],['found','found'],['you','you'],['look','looking'],['king','looking'],['back','back']]
'''
# lyrics = [['You','You'],['turn','turn'],['my','my'],['nights','nights'],
#           ['in','into'],['to','into'],['days','days'],['Lead','Lead'],['me','me'],['mys','mysterious'],['te','mysterious'],
#           ['ri','mysterious'],['ous','mysterious'],['ways','ways']]
lyrics = [['Then','Then'],['the','the'],['rain','rainstorm'],['storm','rainstorm'],['came','came'],
          ['ov','over'],['er','over'],['me','me'],['and','and'],['i','i'],['felt','felt'],['my','my'],
          ['spi','spirit'],['rit','spirit'],['break','break']]

length_song = len(lyrics)
cond = []

for i in range(20):
    if i < length_song:
        syll2Vec = syllModel.wv[lyrics[i][0]]
        word2Vec = wordModel.wv[lyrics[i][1]]
        cond.append(np.concatenate((syll2Vec,word2Vec)))
    else:
        cond.append(np.concatenate((syll2Vec,word2Vec)))


flattened_cond = []
for x in cond:
    for y in x:
        flattened_cond.append(y)


In [8]:

model_path = './saved_gan_models/saved_model_best_overall_mmd'
# model_path = './saved_gan_models/saved_model_end_of_training'

x_list = []
y_list = []

with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [], model_path)
    graph = tf.get_default_graph()
    keep_prob = graph.get_tensor_by_name("model/keep_prob:0")
    input_metadata = graph.get_tensor_by_name("model/input_metadata:0")
    input_songdata = graph.get_tensor_by_name("model/input_data:0")
    output_midi = graph.get_tensor_by_name("output_midi:0")
    feed_dict = {}
    feed_dict[keep_prob.name] = 1.0
    condition = []
    feed_dict[input_metadata.name] = condition
    feed_dict[input_songdata.name] = np.random.uniform(size=(1, 20, 3))
    condition.append(np.split(np.asarray(flattened_cond), 20))
    feed_dict[input_metadata.name] = condition
    generated_features = sess.run(output_midi, feed_dict)
    sample = [x[0, :] for x in generated_features]
    sample = midi_statistics.tune_song(utils.discretize(sample))
    midi_pattern = utils.create_midi_pattern_from_discretized_data(sample[0:length_song])
    destination = "test.mid"
    midi_pattern.write(destination)
    
    print('done')


INFO:tensorflow:Restoring parameters from ./saved_gan_models/saved_model_best_overall_mmd\variables\variables
done
